1 - Prepare data

In [7]:
import pandas as pd
import numpy as np
import datetime as date

#Choose the death cases data from hospital.csv from March to August
df = pd.read_csv('https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv')
df['date'] = pd.date_range('2020-02-1', periods=948, freq='D')
mask = (df['date'] > '2021-5-31') & (df['date'] <= '2022-1-31')
cases_malaysia = df.loc[mask][['date','cases_active','cases_new']]
print(cases_malaysia)

In [20]:
#Choose the death cases data from deaths_malaysia.csv from March to August
df = pd.read_csv('https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/hospital.csv')
df  = df.groupby('date').sum()
df['date'] = pd.date_range('2020-03-24', periods=890, freq='D')
mask = (df['date'] > '2021-5-31') & (df['date'] <= '2022-1-31')
cases_malaysia = df.loc[mask][['beds_covid','beds','admitted_total','hosp_covid','hosp_pui']]
print(cases_malaysia)

            beds_covid   beds  admitted_total  hosp_covid  hosp_pui
date                                                               
2021-06-01       10730  38285            1994        7676      1280
2021-06-02       10824  38318            1875        7587      1175
2021-06-03       10813  38318            1906        7565      1247
2021-06-04       10871  38318            1833        7337      1191
2021-06-05       10896  38318            1992        7457      1082
...                ...    ...             ...         ...       ...
2022-01-27        9877  40899             964        2002       888
2022-01-28        9794  40827             955        1844       850
2022-01-29        9781  40827             890        2036       912
2022-01-30        9781  40827             901        2070       927
2022-01-31        9723  40837             864        2064       906

[245 rows x 5 columns]


In [21]:
icu_initial = pd.read_csv('https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/icu.csv')
icu_initial  = icu_initial.groupby('date').sum()
icu_initial['date'] = pd.date_range('2020-03-24',periods=890, freq='D')
mask = (icu_initial['date'] > '2021-5-31') & (icu_initial['date'] <= '2022-1-31')
icu = icu_initial.loc[mask][['icu_covid','beds_icu_covid','icu_pui']]
icu

,icu_covid,beds_icu_covid,icu_pui
date,,,
2021-06-01,1181,1126,9
2021-06-02,1202,1124,11
2021-06-03,1176,1124,14
2021-06-04,1154,1126,11
2021-06-05,1155,1132,12
...,...,...,...
2022-01-27,124,859,0
2022-01-28,127,859,0
2022-01-29,119,859,0


In [22]:
print(cases_malaysia)
print(icu)

            beds_covid   beds  admitted_total  hosp_covid  hosp_pui
date                                                               
2021-06-01       10730  38285            1994        7676      1280
2021-06-02       10824  38318            1875        7587      1175
2021-06-03       10813  38318            1906        7565      1247
2021-06-04       10871  38318            1833        7337      1191
2021-06-05       10896  38318            1992        7457      1082
...                ...    ...             ...         ...       ...
2022-01-27        9877  40899             964        2002       888
2022-01-28        9794  40827             955        1844       850
2022-01-29        9781  40827             890        2036       912
2022-01-30        9781  40827             901        2070       927
2022-01-31        9723  40837             864        2064       906

[245 rows x 5 columns]
            icu_covid  beds_icu_covid  icu_pui
date                                         